In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import logging
import pickle
import json
import subprocess
import cv2

from src.agent.planner import get_plan_with_llm
from src.agent.director import get_segment_guidance
from src.agent.editor import generate_segment_video, EditResult
from src.agent.music_utils import analyze_music, get_music_info
from src.agent.editing_utils import Score, ScoreConfig, load_video_footages, ShotCandidate
from src.agent.interaction_utils import parse_segment_guidance

/home/keli/miniconda3/envs/videoedit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - [%(name)s] %(message)s'))

file_handler = logging.FileHandler("tmp.log", mode='w')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - [%(name)s] - %(message)s'))

planner_logger = logging.getLogger('planner')
planner_logger.setLevel(logging.DEBUG)
planner_logger.addHandler(console_handler)
planner_logger.addHandler(file_handler)
planner_logger.propagate = False

editor_logger = logging.getLogger('editor')
editor_logger.setLevel(logging.DEBUG)
editor_logger.addHandler(console_handler)
editor_logger.addHandler(file_handler)
editor_logger.propagate = False

director_logger = logging.getLogger('director')
director_logger.setLevel(logging.DEBUG)
director_logger.addHandler(console_handler)
director_logger.addHandler(file_handler)
director_logger.propagate = False

retriever_logger = logging.getLogger('retriever')
retriever_logger.setLevel(logging.DEBUG)
retriever_logger.addHandler(console_handler)
retriever_logger.addHandler(file_handler)
retriever_logger.propagate = False

In [3]:
# --- Test Input 1: User Intent ---
user_prompt = """
I want an adrenaline-pumping Mission Impossible tribute. 
The theme is 'The Art of the Impossible'. 
Start with high tension and silence, building up to a 
chaotic climax with heavy action, and ending with a heroic silhouette.
"""

# --- Test Input 2: VLM-style Footage Summary ---
# Resembling raw output from a video-language model
footage_summary = """
Footage Analysis Report: Action Thriller Collection
Visual Style & Tone High-contrast aesthetics with a shift between cold blue/grey tech environments and warm orange/yellow desert and explosion scenes. Extremely dynamic camera movements including high-altitude aerials, rapid zooms, and handheld tracking shots.
Global Keywords Sprinting, High Altitude, Suspension, Motorcycle, Explosion, Mask, Tactical Gear, Skyscraper, Pursuit, Combat, Burning Fuse, Precision.
Visual Clusters for Retrieval
The Sprint: A character running full speed. Side-profile tracking shots with high motion blur and a sense of extreme urgency.
Vertical Suspension: Figures hanging from extreme heights (buildings, cliffs, planes). High vertical depth, looking down at distant ground, windy environments.
Rapid Pursuit: High-speed vehicle chases involving bikes or cars. Includes drifting, debris, tire smoke, and low-flying aerial support.
Infiltration & Tech: Interaction with screens and terminals. Close-ups of hands operating devices, sweat dripping, and the physical peeling of masks to reveal faces.
Close Combat: Fast-paced hand-to-hand fighting in confined spaces. Focus on limb impact, environmental destruction (glass breaking), and gritty, handheld camera work.
Countdown Motif: Macro shots of a burning fuse with bright sparks against a dark background, symbolizing a ticking clock.
"""

music_prof = analyze_music(Path("../data/musics/02 - The Best Of Me.mp3"))
music_info = get_music_info(music_prof)

In [4]:
print("🚀 Starting Planner Agent Test...")

class PlanResult:
    def __init__(self, narrative, plan, raw_response):
        self.narrative = narrative
        self.plan = plan
        self.raw_response = raw_response

cache_path = "tmp_planner_result.pkl"
if os.path.exists(cache_path):
    print(f"Loading cached planner result from {cache_path}...")
    with open(cache_path, "rb") as f:
        cached_result = pickle.load(f)
    
    print("\n✅ Loaded Cached Result!")
    narrative = cached_result.narrative
    plan = cached_result.plan
    raw_response = cached_result.raw_response
else:
    # Calling your encapsulated function
    narrative, plan, raw_response = get_plan_with_llm(
        user_prompt=user_prompt,
        footage_summary=footage_summary,
        music_info=music_info,  # Ensure this object is in your namespace
        retry_count=3
    )
    cached_result = PlanResult(narrative, plan, raw_response)
    with open(cache_path, "wb") as f:
        pickle.dump(cached_result, f)
    print(f"\n✅ Cached planner result to {cache_path}.")

if plan:
    print("\n✅ Generation Successful!")
    print("-" * 30)
    print(f"TITLE: {user_prompt[:50]}...")
    print("-" * 30)
    print(f"GLOBAL NARRATIVE:\n{narrative}\n")
    
    print(f"{'Section':<15} | {'Energy':<8} | {'Tags'}")
    print("-" * 60)
    for section in plan:
        tags_str = ", ".join(section.get('visual_tags', []))
        print(f"{section.get('section_name', 'N/A'):<15} | "
                f"{section.get('energy_level', 'N/A'):<8} | "
                f"{tags_str}")

    print("\nRAW LLM RESPONSE:")
    print(raw_response)
else:
    print("❌ Failed to generate or parse the plan after retries.")


🚀 Starting Planner Agent Test...
Loading cached planner result from tmp_planner_result.pkl...

✅ Loaded Cached Result!

✅ Generation Successful!
------------------------------
TITLE: 
I want an adrenaline-pumping Mission Impossible t...
------------------------------
GLOBAL NARRATIVE:
The video opens with a tense, silent countdown, establishing the stakes and the protagonist’s isolation. It gradually builds momentum through escalating action sequences — from stealthy infiltration to high-speed chases and vertical suspension stunts — culminating in a chaotic, explosive climax that embodies the “Art of the Impossible.” The final moments transition to a serene, heroic silhouette against a vast, cinematic backdrop, symbolizing triumph and legacy. The narrative arc mirrors the music’s structure: restrained intensity → explosive release → reflective resolution.

Section         | Energy   | Tags
------------------------------------------------------------
verse (0.00s - 20.42s) | Low      | 

In [5]:
# cache guidance_list
cache_path = "tmp_director_guidance.pkl"

if os.path.exists(cache_path):
    print(f"Loading cached director guidance from {cache_path}...")
    with open(cache_path, "rb") as f:
        guidance_list = pickle.load(f)
    print("\n✅ Loaded Cached Director Guidance!")

else:
    print("🚀 Starting Director Agent Test...")

    beats = list(zip(music_prof.beats, music_prof.beat_positions))
    sections = [
        (section, [(beat, position) for beat, position in beats if section.start <= beat < section.end])
        for section in music_prof.segments
    ]

    prev_queries = []

    guidance_list = []

    for idx, curr_seg in enumerate(sections):
        section, beat_info = curr_seg
        print(f"Section {section.label}: {len(beat_info)} beats")
        section_info = plan[idx]  # Match segment index with plan index
        beats_remaining = len(beat_info)
        while (beats_remaining > 0):
            segment_guidance, raw_response = get_segment_guidance(
                user_prompt=user_prompt,
                footage_summary=footage_summary,
                music_info=music_info,
                section_info=section_info,
                beats_remaining=beats_remaining,
                prev_queries=prev_queries,
            )
            print(f"\nGuidance for Section {section.label} with {beats_remaining} beats remaining:")
            print(raw_response[0], raw_response[1], raw_response[2])

            guidance_list.append(segment_guidance)

            beats_remaining -= sum(segment_guidance.get('pacing_control', []))

            prev_queries.append(segment_guidance.get('retrieval_query', ''))
            if len(prev_queries) > 5:
                prev_queries = prev_queries[-5:]

    print("\n✅ Director Guidance Generation Complete!")

    with open(cache_path, "wb") as f:
        pickle.dump(guidance_list, f)

Loading cached director guidance from tmp_director_guidance.pkl...

✅ Loaded Cached Director Guidance!


In [6]:
music_path = Path("/home/keli/VideoEdit/data/musics/02 - The Best Of Me.mp3")
csv_path = Path("/home/keli/VideoEdit/data/csv/MissionImpossible.csv")
data_root = Path("/home/keli/VideoEdit/data/videos/")
output_root = Path("/home/keli/VideoEdit/output/videos/")

video_paths, video_features = load_video_footages(csv_path, data_root, output_root)

In [8]:
beats = music_prof.beats
beats.append(music_prof.segments[-1].end)
curr_beat_idx = 0

result = EditResult(
    video_candidates=[],
    total_frames=0,
    total_score=0.0
)

# Add initial black screen segment
result.append_shot(ShotCandidate(
    video_id="BLACK_SCREEN",
    shot_idx=-1,
    start_frame=0,
    end_frame=round(beats[0] * 24)-1,  
    score=Score(0, 0, 0, 0, 0, 0)
))

for segment_guidance in guidance_list:
    print(f"Generating video for segment with guidance: ")
    print(segment_guidance['retrieval_query'], segment_guidance["weight_profile"], segment_guidance["visual_energy"], segment_guidance['pacing_control'])
    
    prompt_embed, pacing_control, config = parse_segment_guidance(segment_guidance)

    cut_points = []
    for len_beat in pacing_control:
        curr_beat_idx += len_beat
        cut_points.append(beats[curr_beat_idx])

    new_segment = generate_segment_video(video_features, prompt_embed, cut_points, config, result)[0]
    result.extend(new_segment)

result.generate_video(video_paths, Path("tmp.mp4"))

2026-01-18 17:41:51,050 - INFO - [editor] 为段落生成视频，剪辑点：[9.76, 15.1]，已完成帧数：107。


Generating video for segment with guidance: 
Hand adjusting tactical gear close-up Semantic_Priority 45 [8, 8]


2026-01-18 17:41:51,785 - DEBUG - [retriever] 检索相似片段，当前shots数：1，候选池大小：300，需要的片段长度：127，返回Top-3
2026-01-18 17:41:51,786 - DEBUG - [retriever] shots_list为空，使用 None 检索
2026-01-18 17:41:51,812 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:41:51,812 - DEBUG - [retriever] 检索相似片段，当前shots数：2，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:41:51,847 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:41:51,848 - DEBUG - [retriever] 检索相似片段，当前shots数：2，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:41:51,881 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:41:51,882 - DEBUG - [retriever] 检索相似片段，当前shots数：2，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:41:51,915 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:41:51,916 - INFO - [editor] 段落视频生成完成，生成了 5 个候选结果。
2026-01-18 17:41:51,926 - INFO - [editor] 为段落生成视频，剪辑点：[20.42]，已完成帧数：362。


Generating video for segment with guidance: 
A figure crouching in shadows Semantic_Priority 30 [8]


2026-01-18 17:41:52,657 - DEBUG - [retriever] 检索相似片段，当前shots数：3，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:41:52,839 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:41:52,841 - INFO - [editor] 段落视频生成完成，生成了 3 个候选结果。
2026-01-18 17:41:52,852 - INFO - [editor] 为段落生成视频，剪辑点：[25.76, 31.09]，已完成帧数：490。


...

2026-01-18 17:43:36,043 - DEBUG - [retriever] 检索相似片段，当前shots数：109，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:43:36,192 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:43:36,193 - INFO - [editor] 段落视频生成完成，生成了 3 个候选结果。
2026-01-18 17:43:36,203 - INFO - [editor] 为段落生成视频，剪辑点：[260.41, 265.75]，已完成帧数：6122。


Generating video for segment with guidance: 
Silhouette against fading sky Semantic_Priority 30 [8, 8]


2026-01-18 17:43:36,940 - DEBUG - [retriever] 检索相似片段，当前shots数：110，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:43:37,236 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:43:37,237 - DEBUG - [retriever] 检索相似片段，当前shots数：111，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:43:37,532 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:43:37,533 - DEBUG - [retriever] 检索相似片段，当前shots数：111，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:43:37,826 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:43:37,827 - DEBUG - [retriever] 检索相似片段，当前shots数：111，候选池大小：300，需要的片段长度：128，返回Top-3
2026-01-18 17:43:38,119 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:43:38,120 - INFO - [editor] 段落视频生成完成，生成了 5 个候选结果。
2026-01-18 17:43:38,130 - INFO - [editor] 为段落生成视频，剪辑点：[274.0]，已完成帧数：6378。


Generating video for segment with guidance: 
Hero ascending into fading light Semantic_Priority 30 [1]


2026-01-18 17:43:38,881 - DEBUG - [retriever] 检索相似片段，当前shots数：112，候选池大小：300，需要的片段长度：198，返回Top-3
2026-01-18 17:43:39,216 - DEBUG - [retriever] 排序返回Top-3
2026-01-18 17:43:39,217 - INFO - [editor] 段落视频生成完成，生成了 3 个候选结果。
OpenCV: FFMPEG: tag 0x34363268/'h264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616aa1d69d40] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab59f1700] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab59f1700] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab59f1700] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab4936d00] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab4936d00] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab4936d00] Referenced QT chapter track not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x616ab4936d00] Referenc

In [9]:
command = [
    "ffmpeg", "-y", "-i", "tmp.mp4", "-i", music_path, "-c:v", "copy", "-c:a", "aac", "-strict", "experimental", "output.mp4"
]

subprocess.run(command, check=True)

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.3.0 (conda-forge gcc 14.3.0-5)
  configuration: --prefix=/home/keli/miniconda3/envs/videoedit --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1758923993009/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1758923993009/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1758923993009/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1758923993009/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-alsa --enable-libpulse --enable-libvpl --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx26

CompletedProcess(args=['ffmpeg', '-y', '-i', 'tmp.mp4', '-i', PosixPath('/home/keli/VideoEdit/data/musics/02 - The Best Of Me.mp3'), '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', 'output.mp4'], returncode=0)